In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Scrape the betting odds for each team for the day
url = 'https://sports.yahoo.com/nhl/odds/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.find_all('table')
data = []

for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)

df = pd.DataFrame(data)
filtered_df = df[df[0].astype(str).map(len) > 5]

filtered_df['Team'] = filtered_df[0].apply(lambda x: x.split('(')[0].strip())

filtered_df['Odds'] = filtered_df[1].apply(lambda x: x[-4:].strip())

new_df = filtered_df[['Team', 'Odds']]

new_df  

C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3064536699.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Team'] = filtered_df[0].apply(lambda x: x.split('(')[0].strip())
C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3064536699.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Odds'] = filtered_df[1].apply(lambda x: x[-4:].strip())


,Team,Odds
25,Toronto Maple Leafs,-120
26,New York Islanders,+100
29,Arizona Coyotes,+105
30,Buffalo Sabres,-130
33,Detroit Red Wings,+170
34,Dallas Stars,-210
37,Calgary Flames,+150
38,Colorado Avalanche,-185


In [13]:
# use the mapping dictionary to replace the team names with the abbreviations
abbreviations_df = pd.read_csv('teams.csv')
mapping_dict = dict(zip(abbreviations_df['Team'], abbreviations_df['Abbrv']))
new_df['Team'] = new_df['Team'].map(mapping_dict)
new_df


C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3934984110.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Team'] = new_df['Team'].map(mapping_dict)


,Team,Odds
25,TOR,-120
26,NYI,+100
29,ARI,+105
30,BUF,-130
33,DET,+170
34,DAL,-210
37,CGY,+150
38,COL,-185


In [19]:
#convert the american odds into decimal odds, and then into implied probabilities
def convert_american_to_decimal(american_odds):
    if american_odds.startswith('+'):
        return 1 + int(american_odds[1:]) / 100
    elif american_odds.startswith('-'):
        return 1 + 100 / abs(int(american_odds))
    else:
        return None


new_df['Decimal Odds'] = new_df['Odds'].apply(convert_american_to_decimal)
new_df['Decimal Odds'] = new_df['Decimal Odds'].round(2)

def decimal_to_implied_probability(decimal_odds):
    if decimal_odds > 0:
        return round(1 / decimal_odds, 2)
    else:
        return None 

new_df['Implied Probability'] = new_df['Decimal Odds'].apply(decimal_to_implied_probability)
new_df.to_csv('nhl_odds.csv', index=False)


C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3383840344.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Decimal Odds'] = new_df['Odds'].apply(convert_american_to_decimal)
C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3383840344.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Decimal Odds'] = new_df['Decimal Odds'].round(2)
C:\Users\luken\AppData\Local\Temp\ipykernel_11484\3383840344.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,Team,Odds,Decimal Odds,Implied Probability
25,TOR,-120,1.83,0.55
26,NYI,+100,2.00,0.50
29,ARI,+105,2.05,0.49
30,BUF,-130,1.77,0.56
33,DET,+170,2.70,0.37
34,DAL,-210,1.48,0.68
37,CGY,+150,2.50,0.40
38,COL,-185,1.54,0.65
